In [ ]:
%load_ext autoreload
%autoreload 

In [ ]:
import online_retail_analysis.utils.paths as path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans,MiniBatchKMeans, AgglomerativeClustering
from sklearn.preprocessing import OneHotEncoder
from scipy import stats

sns.set_style("whitegrid")
%matplotlib inline
pd.options.display.float_format = '{:,.3f}'.format

### Import data

In [ ]:
df = pd.read_excel(path.data_raw_dir('Online Retail.xlsx')) 
#encoding = 'unicode_escape', engine ='python'
df.sample(5)

### Check data

In [ ]:
df.info()#see the information

In [ ]:
df_proces = df.copy() # create a copy the data
df_proces.isna().sum() # see the number of row whit NaN

In [ ]:
df_proces[df_proces.isnull().any(axis=1)] # see the null data by columns

In [ ]:
df_proces.isnull().any(axis=0) # see null data from rows

In [ ]:
df_proces.isnull().sum() # see all null data

In [ ]:
df_proces = df_proces[pd.notnull(df_proces['CustomerID'])] # delete null data from Customer ID

In [ ]:
df_proces.InvoiceNo.value_counts()

In [ ]:
df_proces.duplicated().sum() #see the number of row duplicated

In [ ]:

df_proces.drop_duplicates(inplace=True) # remove the duplicated

df_proces.describe() # look the a little statistics 


In [ ]:
# delete Cancel transaccion 
df_proces['InvoiceNo'] = df_proces['InvoiceNo'].astype('str')
df_proces = df_proces[~df_proces['InvoiceNo'].str.contains('C')]

In [ ]:
# delete Price and Quantity less than 0
df_proces = df_proces[df_proces.Quantity > 0]
df_proces = df_proces[df_proces.UnitPrice > 0]

In [ ]:
print('Number of customers: {}'.format(len(df_proces.CustomerID.unique())))

print('Number of customers: {}'.format(len(df_proces.Country.unique())))

# df_proces.Country.value_counts()


In [ ]:
df_proces['date']= pd.to_datetime(df_proces['InvoiceDate'])
df_proces['month'] = df_proces['date'].dt.month
df_proces['days'] = df_proces['date'].dt.day_name()
df_proces

### Remove outliers from price and quantity

In [ ]:
fig, axes= plt.subplots(1,2,figsize=(12,8))
sns.boxplot(y='Quantity', data=df_proces, ax=axes[0])
sns.boxplot(y='UnitPrice', data=df_proces, ax=axes[1])
plt.show()

In [ ]:
for col in ['Quantity','UnitPrice']:
    p_05 = df_proces[col].quantile(0.05) # 5th quantile
    p_95 = df_proces[col].quantile(0.95) # 95th quantile
    df_proces[col].clip(p_05, p_95, inplace=True)
fig, axes= plt.subplots(1,2,figsize=(12,8))
sns.boxplot(y='Quantity', data=df_proces, ax=axes[0])
sns.boxplot(y='UnitPrice', data=df_proces, ax=axes[1])
plt.show()

### Export Data Clean

In [ ]:
df_proces.to_csv(path.data_processed_dir('data_clean.csv'),index=False)


But how had the year? 